# Making Compound Lenses
This notebook describes how the gravitational compound lenses were created in [1]. 




Faber-Jackson relationship magnitude to mass.
We use magnitute and redshift pairs from the single lens dataset. 

In [1]:
# standard imports
import copy
import os
import numpy as np
import pandas as pd
import random
from PIL import Image
import matplotlib.pyplot as plt

# skypy imports
import skypy
from skypy.pipeline import Pipeline

import astropy
from astropy.modeling import models

# lenstronomy module import
from lenstronomy.SimulationAPI.sim_api import SimAPI
import lenstronomy.Plots.plot_util as plot_util
from lenstronomy.Util import constants
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
import lenstronomy.Util.data_util as data_util
import lenstronomy.Util.util as util
from lenstronomy.LightModel.Profiles.gaussian import GaussianEllipse

## Faber Jackson

Data is loaded from Table 1 in [2]. This is used to calculate the velocity dispersion from the magnitude. 

In [6]:
def give_me_a_straight_line(x,y):
        w, b  = np.polyfit(x,y,deg=1)
        return w,b 

def MakeFaberJackson():
    FJG = pd.read_csv('/Volumes/Seagate Backup Plus Drive/CompoundLensModelling/FaberJacksonGraph.csv')

    X = FJG['MB (Mag)'].values
    Y = np.log10(FJG['Fourier Velocity 1'].values)

    w, b = give_me_a_straight_line(X,Y)

    return X,Y,w,b
    #y = w * X + b

## CCD Specifications
The information for the Euclid camera and band_obs are taken from Lenstronomy. Small changes are made to make the compound lens dataset as similiar to the single lens dataset as possible.

In [7]:
gauss = GaussianEllipse()
Euclid_camera_NISP = {'read_noise': 10,  # std of noise generated by read-out (in units of electrons)
               'pixel_scale': 0.3,  # scale (in arcseonds) of pixels
               'ccd_gain': 4.5  # electrons/ADU (analog-to-digital unit). A gain of 8 means that the camera digitizes the CCD signal so that each ADU corresponds to 8 photoelectrons.
              }

Euclid_camera_VIS = {'read_noise': 4.2,  # std of noise generated by read-out (in units of electrons)
               'pixel_scale': 0.1,  # scale (in arcseonds) of pixels
               'ccd_gain': 3.1  # electrons/ADU (analog-to-digital unit). A gain of 8 means that the camera digitizes the CCD signal so that each ADU corresponds to 8 photoelectrons.
              }

Euclid_J_band_obs = {'exposure_time': 264.,  # exposure time per image (in seconds)
                   'sky_brightness': 22.35,  # sky brightness (in magnitude per square arcseconds)
                   'magnitude_zero_point': 24.0,  # magnitude in which 1 count per second per arcsecond square is registered (in ADU's)
                   'num_exposures': 3,  # number of exposures that are combined
                   'seeing': 0.16,  # full width at half maximum of the PSF (if not specific psf_model is specified)
                   'psf_type': 'GAUSSIAN',  # string, type of PSF ('GAUSSIAN' and 'PIXEL' supported)
                   'kernel_point_source': None  # 2d numpy array, model of PSF centered with odd number of pixels per axis (optional when psf_type='PIXEL' is chosen)
                  }

Euclid_Y_band_obs = {'exposure_time': 264.,
                   'sky_brightness': 22.35,
                   'magnitude_zero_point': 24.0,
                   'num_exposures': 3,
                   'seeing': 0.16,
                   'psf_type': 'GAUSSIAN'}

Euclid_H_band_obs = {'exposure_time': 264.,
                   'sky_brightness': 22.35,
                   'magnitude_zero_point': 24.0,
                   'num_exposures': 3,
                   'seeing': 0.16,
                   'psf_type': 'GAUSSIAN'}

Euclid_VIS_band_obs = {'exposure_time': 1800.,
                   'sky_brightness': 22.35,
                   'magnitude_zero_point': 24.0,
                   'num_exposures': 3,
                   'seeing': 0.16,
                   'psf_type': 'GAUSSIAN'}

# here we merge observations and instrument for the three bands
kwargs_J_band = util.merge_dicts(Euclid_camera_NISP, Euclid_J_band_obs)
kwargs_Y_band = util.merge_dicts(Euclid_camera_NISP, Euclid_Y_band_obs)
kwargs_H_band = util.merge_dicts(Euclid_camera_NISP, Euclid_H_band_obs)
kwargs_VIS_band = util.merge_dicts(Euclid_camera_VIS, Euclid_VIS_band_obs)

kwargs_numerics = {'point_source_supersampling_factor': 1}

## Making the Compound Lenses

The gravitational lensing process, this function generates the compound lens system using the variables provided to it.


In [8]:
def CompoundLensMaker2(kwargs_model_composite, kwargs_lens, kwargs_lens_light_mag_VIS, kwargs_source_mag_VIS, kwargs_lens_light_mag_J, kwargs_source_mag_J, kwargs_lens_light_mag_Y, 
                      kwargs_source_mag_Y, kwargs_lens_light_mag_H, kwargs_source_mag_H):
    # we only simulate here the g-band, see above how to combine multiple bands
    kwargs_numerics = {'point_source_supersampling_factor': 1}
    kwargs_ps_mag_VIS = [{'magnitude': 13, 'ra_source': 0.03, 'dec_source': -30}]
    
    sim_VIS = SimAPI(numpix=200, kwargs_single_band=kwargs_VIS_band, kwargs_model=kwargs_model_composite)
    imSim_VIS = sim_VIS.image_model_class(kwargs_numerics)
    sim_J = SimAPI(numpix=66, kwargs_single_band=kwargs_J_band, kwargs_model=kwargs_model_composite)    
    imSim_J = sim_J.image_model_class(kwargs_numerics)
    sim_Y = SimAPI(numpix=66, kwargs_single_band=kwargs_Y_band, kwargs_model=kwargs_model_composite)
    imSim_Y = sim_Y.image_model_class(kwargs_numerics)
    sim_H = SimAPI(numpix=66, kwargs_single_band=kwargs_H_band, kwargs_model=kwargs_model_composite)
    imSim_H = sim_H.image_model_class(kwargs_numerics)

# Y-band
    V_Y_ps = 0
    kwargs_ps_mag_Y = copy.deepcopy(kwargs_ps_mag_VIS)
    kwargs_ps_mag_Y[0]['magnitude'] -= V_Y_ps

# J-band
    V_J_ps = 0
    kwargs_ps_mag_J = copy.deepcopy(kwargs_ps_mag_VIS)
    kwargs_ps_mag_J[0]['magnitude'] -= V_J_ps

# H-band
    V_H_ps = 0
    kwargs_ps_mag_H = copy.deepcopy(kwargs_ps_mag_VIS)
    kwargs_ps_mag_H[0]['magnitude'] -= V_H_ps

# turn magnitude kwargs into lenstronomy kwargs
    kwargs_lens_light_VIS, kwargs_source_VIS, kwargs_ps_VIS = sim_VIS.magnitude2amplitude(kwargs_lens_light_mag_VIS, kwargs_source_mag_VIS, kwargs_ps_mag_VIS)
    kwargs_lens_light_Y, kwargs_source_Y, kwargs_ps_Y = sim_Y.magnitude2amplitude(kwargs_lens_light_mag_Y, kwargs_source_mag_Y, kwargs_ps_mag_Y)
    kwargs_lens_light_J, kwargs_source_J, kwargs_ps_J = sim_J.magnitude2amplitude(kwargs_lens_light_mag_J, kwargs_source_mag_J, kwargs_ps_mag_J)
    kwargs_lens_light_H, kwargs_source_H, kwargs_ps_H = sim_H.magnitude2amplitude(kwargs_lens_light_mag_H, kwargs_source_mag_H, kwargs_ps_mag_H)
    
    image_VIS = imSim_VIS.image(kwargs_lens, kwargs_source_VIS, kwargs_lens_light_VIS)
    image_J = imSim_J.image(kwargs_lens, kwargs_source_J, kwargs_lens_light_J)
    image_Y = imSim_Y.image(kwargs_lens, kwargs_source_Y, kwargs_lens_light_Y)
    image_H = imSim_H.image(kwargs_lens, kwargs_source_H, kwargs_lens_light_H)
       
    image_VIS += sim_VIS.noise_for_model(model=image_VIS)
    image_J += sim_J.noise_for_model(model=image_J)
    image_Y += sim_Y.noise_for_model(model=image_Y)
    image_H += sim_H.noise_for_model(model=image_H)
    
    from skimage.transform import resize

    img_VIS = np.zeros((200,200, 1), dtype=float)
    img_NISP = np.zeros((66,66, 3), dtype=float)
    img_VIS = plot_util.sqrt(image_VIS, scale_min=0, scale_max=10000)
    img_NISP[:,:,0] = plot_util.sqrt(image_Y, scale_min=0, scale_max=10000)
    img_NISP[:,:,1] = plot_util.sqrt(image_J, scale_min=0, scale_max=10000)
    img_NISP[:,:,2] = plot_util.sqrt(image_H, scale_min=0, scale_max=10000)

    return img_VIS, img_NISP

## Make Galaxies
Using the redshift and mangitude pairs from the single lens dataset. The apparent magnitude for each band is generated using SkyPy. This is used to make Blue source galaxies and Red lens galaxies.

In [9]:
def MakeBlueGalaxies(redshift,M_lens, cosmology):
    alpha0= [2.079, 3.524, 1.917, 1.992, 2.536]
    alpha1= [2.265, 3.862, 1.921, 1.685, 2.480]
    weight= [3.47e+09, 3.31e+06, 2.13e+09, 1.64e+10, 1.01e+09]
    spectral_coefficients = skypy.galaxies.spectrum.dirichlet_coefficients(redshift, alpha0,alpha1,weight)
    filter= 'Euclid-VIS','Euclid-Y','Euclid-H','Euclid-J'
    M = M_lens - cosmo.distmod(redshift).value
    stellar_mass = skypy.galaxies.spectrum.kcorrect.stellar_mass(spectral_coefficients,M,filter)
    app_mag = skypy.galaxies.spectrum.kcorrect.apparent_magnitudes( spectral_coefficients, redshift, filter,cosmology, stellar_mass=stellar_mass)
    
    return app_mag

In [10]:
def MakeRedGalaxies(redshift, M_lens,cosmology):
    alpha0= [2.461, 2.358, 2.568, 2.268, 2.402]
    alpha1= [2.410, 2.340, 2.200, 2.540, 2.464]
    weight = [3.84e+09, 1.57e+06, 3.91e+08, 4.66e+10, 3.03e+07]
    spectral_coefficients = skypy.galaxies.spectrum.dirichlet_coefficients(redshift, alpha0,alpha1,weight)
    filter= 'Euclid-VIS','Euclid-Y','Euclid-H','Euclid-J'
    M = M_lens - cosmo.distmod(redshift).value
    stellar_mass = skypy.galaxies.spectrum.kcorrect.stellar_mass(spectral_coefficients,M,filter)
    app_mag = skypy.galaxies.spectrum.kcorrect.apparent_magnitudes( spectral_coefficients, redshift, filter, cosmology, stellar_mass=stellar_mass)
    
    return app_mag

## Sampling Magnitudes and redshifts
This randomly selects magnitude and redshift pairs from the single lens dataset. The lens galaxies are arranged to ensure that the lenses appear in the correct redshift order.

In [11]:
def Generate_Lens_System_Mag_Z_2():
    index2 = np.random.randint(0,100000)

    Z3_Data_A = csv[csv['z_source']>3]
    Z3_Data_L = csv[csv['z_source']<3]
    
    index0 = np.random.randint(0,Z3_Data_A.shape[0])
    index1 = np.random.randint(0,Z3_Data_L.shape[0])
    
    z_source = Z3_Data_A['z_source'].values[index0]
    mag_source = Z3_Data_A['mag_source'].values[index0]
    
    Z_lens_1 = Z3_Data_L['z_source'].values[index1]
    mag_lens_1 = Z3_Data_L['mag_source'].values[index1]

    Z_lens_2 = csv['z_lens'].values[index2]
    mag_lens_2 = csv['mag_lens'].values[index2]
    
    if Z_lens_1 > Z_lens_2: 
        Mid_lens_Z = Z_lens_1
        Mid_lens_Mag = mag_lens_1
        Edge_lens_Z = Z_lens_2
        Edge_lens_Mag = mag_lens_2
    else:
        Mid_lens_Z = Z_lens_2
        Mid_lens_Mag = mag_lens_2
        Edge_lens_Z = Z_lens_1
        Edge_lens_Mag = mag_lens_1
        
    return z_source, mag_source, Mid_lens_Z, Mid_lens_Mag, Edge_lens_Z, Edge_lens_Mag

## Theta E Velocity Dispersion
Uses the Faber Jackson relationship to calculate the Einstein radius for the gravitational lens

In [12]:
def Get_Theta_E_From_Velo_Dist(mag_lens, Z_lens, z_source):
    X,Y,w,b = MakeFaberJackson()
    log_vel_d = w * (mag_lens - cosmo.distmod(Z_lens).value) + b
    vel_disp_iso =10**log_vel_d
    lens_cosmo = LensCosmo(Z_lens, z_source, cosmo=cosmo)
    theta_E_SIS = lens_cosmo.sis_sigma_v2theta_E(vel_disp_iso)
    
    return theta_E_SIS

In [13]:
csv = pd.read_csv('/Volumes/Seagate Backup Plus Drive/EuclidDataFile2.0train.csv')
#csv = pd.read_csv('F:/EuclidDataFile2.0train.csv')

## Generate lens positions
THis generates the position of galaxies within the image when given a position and an Einstein Radius. 

In [14]:
def generateLensPos(X,Y, E_theta):
    x1 = np.random.uniform(-E_theta/2, E_theta/2)
    y1 = np.random.uniform(-E_theta/2, E_theta/2)
    
    if X + x1 >=5 or X + x1 <= -5:
        X_lens = X - x1
    else:
        X_lens = X + x1
        
    if Y + y1 >=5 or Y + y1 <= -5:
        Y_lens = Y - y1
    else:
        Y_lens = Y + y1
        
    return X_lens, Y_lens

## Generating angles
This converts the angle and the axis ratio of the galaxy into e1 and e2 parameters

In [15]:
def phi_q2_ellipticity(phi, q):
    e1 = (1 - q) / (1 + q) * np.cos(2 * phi)
    e2 = (1 - q) / (1 + q) * np.sin(2 * phi)    
    
    return e1, e2

## specify cosmos

In [16]:
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.)

## Generating the compound lenses
This process generates everything needed to make compound lenses

This gets three magnitude and redshift pairs for the galaxies in the image, then values are calculated and sampled for each variable needed to generate the compound lenses. If there is a problem at this stage every variable is given the value 1138 and countMod = 0. This provides a clear way to know if there was a problem. 
The output from this method is the NISP and VIS band images, the variables used to generate the compound lenses, and the countMod variable.


In [23]:
def CreateCompundLenses(cosmo=cosmo):
    Edge_source_Z, Edge_source_Mag, Mid_lens_Z, Mid_lens_Mag, Edge_lens_Z, Edge_lens_Mag = Generate_Lens_System_Mag_Z_2()
    
    try:
        Edge_Lens_Gal_Mag = MakeRedGalaxies(Edge_lens_Z,Edge_lens_Mag, cosmo)
        Mid_Lens_Gal_Mag = MakeRedGalaxies(Mid_lens_Z,Mid_lens_Mag, cosmo)
        Edge_Source_Gal_Mag = MakeBlueGalaxies(Edge_source_Z, Edge_source_Mag, cosmo)

        theta_E_SIS_Mid = Get_Theta_E_From_Velo_Dist(Mid_lens_Mag, Mid_lens_Z, Edge_source_Z)
        theta_E_SIS_EdgeML =  Get_Theta_E_From_Velo_Dist(Edge_lens_Mag, Edge_lens_Z, Mid_lens_Z)
        theta_E_SIS_EdgeES = Get_Theta_E_From_Velo_Dist(Edge_lens_Mag, Edge_lens_Z, Edge_source_Z)

        R_sersic_Edge_Source = np.random.uniform(0.2, 7)
        n_sersic_Edge_Source = np.random.uniform(1.5, 4)
        centre_X_Edge_Source = 0 #np.random.uniform(-5, 5)
        centre_Y_Edge_Source = 0 #np.random.uniform(-5, 5)

        R_sersic_Mid_Lens = np.random.uniform(0.2, 7)
        n_sersic_Mid_Lens = np.random.uniform(1.5, 4)
        phi_Mid_Lens = np.random.uniform(0, np.pi)
        q_Mid_Lens =  np.random.uniform(0.3, 1)
        e1_Mid_Lens , e2_Mid_Lens = phi_q2_ellipticity(phi_Mid_Lens, q_Mid_Lens)
        centre_X_Mid_Lens = centre_X_Edge_Source + np.random.uniform(-0.25, 0.25)
        centre_Y_Mid_Lens = centre_Y_Edge_Source + np.random.uniform(-0.25, 0.25)#= generateLensPos(centre_X_Edge_Source, centre_Y_Edge_Source, theta_E_SIS_Mid)
        gamma1_Mid_Lens =  np.random.uniform(-0.05, 0.05)
        gamma2_Mid_Lens =  np.random.uniform(-0.05, 0.05)

        R_sersic_Edge_Lens = np.random.uniform(0.2, 7)
        n_sersic_Edge_Lens = np.random.uniform(1.5, 4)
        phi_Edge_Lens =  np.random.uniform(0, np.pi)
        q_Edge_Lens =   np.random.uniform(0.3, 1)
        e1_Edge_Lens , e2_Edge_Lens =  phi_q2_ellipticity(phi_Edge_Lens, q_Edge_Lens)
        centre_X_Edge_Lens = centre_X_Edge_Source + np.random.uniform(-0.25, 0.25)
        centre_Y_Edge_Lens = centre_Y_Edge_Source + np.random.uniform(-0.25, 0.25) #generateLensPos(centre_X_Edge_Source, centre_Y_Edge_Source, theta_E_SIS_Edge)
        gamma1_Edge_Lens = np.random.uniform(-0.05, 0.05)
        gamma2_Edge_Lens = np.random.uniform(-0.05, 0.05)
        

        kwargs_model_composite = {'lens_model_list': ['SIE', 'SHEAR', 'SIE', 'SHEAR'],#, 'SIE', 'SHEAR'],  # list of lens models to be used
                          'lens_redshift_list': [Mid_lens_Z, Mid_lens_Z, Edge_lens_Z, Edge_lens_Z],#, Edge_lens_Z, Edge_lens_Z],  # list of redshift of the deflections
                          'lens_light_model_list': ['SERSIC_ELLIPSE', 'SERSIC_ELLIPSE'],  # list of unlensed light models to be used
                          'source_light_model_list': [ 'SERSIC'],  # list of extended source models to be used
                          'source_redshift_list': [Edge_source_Z],  # list of redshfits of the sources in same order as source_light_model_list
                          'cosmo': cosmo,  # astropy.cosmology instance
                          'z_source_convention': 5,  # source redshfit to which the reduced deflections are computed, is the maximal redshift of the ray-tracing
                          'z_source': 5,  # redshift of the default source (if not further specified by 'source_redshift_list') and also serves as the redshift of lensed point sources
        }

# gamma is the shear component 
        kwargs_lens = [
            {'theta_E': theta_E_SIS_Mid, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},  # SIE model
            {'gamma1': gamma1_Mid_Lens, 'gamma2': gamma2_Mid_Lens},  # SHEAR model
            {'theta_E': theta_E_SIS_EdgeML, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}, # SIE model
            {'gamma1': gamma1_Edge_Lens, 'gamma2': gamma2_Edge_Lens}#,
           # {'theta_E': theta_E_SIS_EdgeES, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}, # SIE model
           # {'gamma1': gamma1_Edge_Lens, 'gamma2': gamma2_Edge_Lens}
        ]
# lens light
        kwargs_lens_light_mag_VIS = [{'magnitude': Mid_Lens_Gal_Mag[0], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                            {'magnitude': Edge_Lens_Gal_Mag[0], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_Y= [{'magnitude': Mid_Lens_Gal_Mag[1], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                            {'magnitude': Edge_Lens_Gal_Mag[1], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_H = [{'magnitude': Mid_Lens_Gal_Mag[2], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                            {'magnitude': Edge_Lens_Gal_Mag[2], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]

        kwargs_lens_light_mag_J = [{'magnitude': Mid_Lens_Gal_Mag[3], 'R_sersic': R_sersic_Mid_Lens, 'n_sersic': n_sersic_Mid_Lens, 'e1': e1_Mid_Lens, 'e2': e2_Mid_Lens, 'center_x': centre_X_Mid_Lens, 'center_y': centre_Y_Mid_Lens},
                            {'magnitude': Edge_Lens_Gal_Mag[3], 'R_sersic': R_sersic_Edge_Lens, 'n_sersic': n_sersic_Edge_Lens, 'e1': e1_Edge_Lens, 'e2': e2_Edge_Lens, 'center_x': centre_X_Edge_Lens, 'center_y': centre_Y_Edge_Lens}]
# source light
        kwargs_source_mag_VIS = [
                      {'magnitude': Edge_Source_Gal_Mag[0], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_Y = [
                      {'magnitude': Edge_Source_Gal_Mag[1], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_H = [
                      {'magnitude': Edge_Source_Gal_Mag[2], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]
        kwargs_source_mag_J = [
                      {'magnitude': Edge_Source_Gal_Mag[3], 'R_sersic': R_sersic_Edge_Source, 'n_sersic': n_sersic_Edge_Source, 'center_x': centre_X_Edge_Source, 'center_y': centre_Y_Edge_Source},]

    
        img_VIS, img_NISP = CompoundLensMaker2(kwargs_model_composite, kwargs_lens, kwargs_lens_light_mag_VIS, kwargs_source_mag_VIS, kwargs_lens_light_mag_J, kwargs_source_mag_J, kwargs_lens_light_mag_Y, 
                      kwargs_source_mag_Y, kwargs_lens_light_mag_H, kwargs_source_mag_H)
    
        countMod = 1

    except ValueError:
        Edge_Lens_Gal_Mag = [1138, 1138, 1138, 1138]
        Mid_Lens_Gal_Mag =  [1138, 1138, 1138, 1138]
        Edge_Source_Gal_Mag =  [1138, 1138, 1138, 1138]

        theta_E_SIS_Mid = 1138
        theta_E_SIS_EdgeML  = 1138
        theta_E_SIS_EdgeES  = 1138

        R_sersic_Edge_Source = 1138
        n_sersic_Edge_Source = 1138
        centre_X_Edge_Source = 1138
        centre_Y_Edge_Source = 1138

        R_sersic_Mid_Lens = 1138
        n_sersic_Mid_Lens = 1138
        e1_Mid_Lens = 1138
        e2_Mid_Lens = 1138
        centre_X_Mid_Lens = 1138
        centre_Y_Mid_Lens = 1138
        gamma1_Mid_Lens = 1138
        gamma2_Mid_Lens = 1138

        R_sersic_Edge_Lens = 1138
        n_sersic_Edge_Lens = 1138
        e1_Edge_Lens = 1138
        e2_Edge_Lens = 1138
        centre_X_Edge_Lens = 1138
        centre_Y_Edge_Lens = 1138
        gamma1_Edge_Lens = 1138
        gamma2_Edge_Lens = 1138
        
        img_VIS =  np.random.random((200,200))
        img_NISP =  np.random.random((66,66,3))
        
        countMod = 0
    
    
    arr_params = [Edge_source_Z, Edge_source_Mag, Mid_lens_Z,  Mid_lens_Mag,  Edge_lens_Z, Edge_lens_Mag,  Edge_Source_Gal_Mag[0], 
                  Edge_Source_Gal_Mag[1], Edge_Source_Gal_Mag[2], Edge_Source_Gal_Mag[3],  Mid_Lens_Gal_Mag[0],  Mid_Lens_Gal_Mag[1], Mid_Lens_Gal_Mag[2],  
                  Mid_Lens_Gal_Mag[3],  Edge_Lens_Gal_Mag[0], Edge_Lens_Gal_Mag[1],Edge_Lens_Gal_Mag[2],  Edge_Lens_Gal_Mag[3], theta_E_SIS_Mid, 
                  theta_E_SIS_EdgeML,theta_E_SIS_EdgeES, R_sersic_Edge_Source,  n_sersic_Edge_Source, centre_X_Edge_Source,  centre_Y_Edge_Source,  R_sersic_Mid_Lens,  
                  n_sersic_Mid_Lens, e1_Mid_Lens, e2_Mid_Lens, centre_X_Mid_Lens,  centre_Y_Mid_Lens, gamma1_Mid_Lens,  gamma2_Mid_Lens, R_sersic_Edge_Lens,  
                  n_sersic_Edge_Lens, e1_Edge_Lens, e2_Edge_Lens, centre_X_Edge_Lens, centre_Y_Edge_Lens, gamma1_Edge_Lens, gamma2_Edge_Lens]
    
    
    
    
    return img_VIS, img_NISP, arr_params, countMod

## Create a single compound lenses

In [24]:
img_VIS, img_NISP, arr_params, countMod = CreateCompundLenses(cosmo=cosmo)

## Making Multiple Compound Lenses

In [51]:
i = 0
no_compoundLenses = 9
loc = 'CompoundLensModelling/'
while i < no_compoundLenses:
#    print(i)
    img_VIS, img_NISP, arr_params, countMod = CreateCompundLenses(cosmo=cosmo)
    np.save(loc+'Y/CompoundLens_'+str(i), img_NISP[:,:,0])
    np.save(loc+'J/CompoundLens_'+str(i), img_NISP[:,:,1])
    np.save(loc+'H/CompoundLens_'+str(i), img_NISP[:,:,2])
    np.save(loc+'VIS/CompoundLens_'+str(i), img_VIS)
    np.save(loc+'Params/arr_params_'+str(i), arr_params)
    i = i + countMod

# References
[1] Detecting gravitational lenses using machine learning: exploring interpretability and sensitivity to rare lensing configurations

